In [ ]:
!pip install pandas-profiling==3.1.0

In [40]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
import warnings
import plotly.graph_objects as go

In [31]:
warnings.filterwarnings('ignore')

In [3]:
np.random.seed = 3050

In [4]:
data_set = pd.read_csv('dataset_customer_churn.csv', sep='^')
data_set = data_set.dropna()

In [ ]:
profile = ProfileReport(data_set)

In [5]:
data_set = data_set.drop(['CD_ASSOCIADO','CODIGO_BENEFICIARIO','REALIZOU_EXODONTIA_COBERTA','REALIZOU_ENDODONTIA_COBERTA', 'A006_REGISTRO_ANS','A006_NM_PLANO','CD_USUARIO','CLIENTE','FORMA_PGTO_MENSALIDADE','QTDE_ATO_N_COBERTO_EXECUTADO','QTDE_ATENDIMENTOS'], axis=1)

In [6]:
y = data_set['SITUACAO']
data_set = data_set.drop(['SITUACAO'], axis=1)

dici_trad = {
    'DESATIVADO' : 0,
    'ATIVO':1
}

y =  y.replace(dici_trad)
y = pd.Series(y)

In [7]:
dummies = pd.get_dummies(data_set[['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE']])

In [8]:
num = data_set.drop(['SEXO','ESTADO_CIVIL','REALIZOU_PROCEDIMEN_ALTO_CUSTO','DIAS_ATE_REALIZAR_ALTO_CUSTO','PLANO','CODIGO_FORMA_PGTO_MENSALIDADE'], axis=1)

In [9]:
x = pd.concat([dummies, num], axis=1)

TESTE E TREINO

In [10]:
x = x
y = y

x_treino,x_teste,y_treino,y_teste = train_test_split(x, y, test_size=0.3, stratify=y)

VIEW FUNCTION

In [11]:
def view_score(y_teste, p) :
  baseline = np.ones(p.shape)

  ac = accuracy_score(y_teste,p)
  pc = precision_score(y_teste,p)
  f1 = f1_score(y_teste,p)

  print("Pelo Accuracy: ",ac * 100,"%")
  print("Pelo Precision: ",pc * 100,"%")
  print("Pelo F1: ",f1 * 100,"%")
  print()
  print("BASELINE")

  ac_b = accuracy_score(y_teste,baseline)
  pc_b = precision_score(y_teste,baseline)
  f1_b = f1_score(y_teste,baseline)

  print("Pelo Accuracy: ",ac_b * 100,"%")
  print("Pelo Precision: ",pc_b * 100,"%")
  print("Pelo F1: ",f1_b * 100,"%")

In [12]:
def cross_validation(x, y, modelo):
  cv = StratifiedKFold(n_splits=5, shuffle=True)

  results = cross_validate(modelo, x, y,cv = cv)
  results = pd.DataFrame(results)
  media = results.test_score.mean() * 100
  desvio = results.test_score.std()
  print("Média Teste: %.2f" %(media),"%")
  print("Entre : [%.2f, %.2f]" %((media - 2 * desvio), (media + 2 * desvio)),"%")

RANDOM FOREST

In [13]:
rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(x_treino, y_treino)

p = rf.predict(x_teste)

view_score(y_teste, p)

Pelo Accuracy:  83.36317168729404 %
Pelo Precision:  87.71163544668588 %
Pelo F1:  80.20009881422925 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


In [14]:
cross_validation(x, y,modelo=rf)

Média Teste: 77.78 %
Entre : [77.64, 77.91] %


LOGISTIC REGRESSION

In [33]:
lr = LogisticRegression(max_iter=50)

lr.fit(x_treino,y_treino)

p2 = lr.predict(x_teste)

view_score(y_teste,p2)

Pelo Accuracy:  81.21567508194735 %
Pelo Precision:  80.07796590446267 %
Pelo F1:  79.17581521999675 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


In [36]:
cross_validation(x,y,modelo=lr)

Média Teste: 80.90 %
Entre : [80.90, 80.91] %


MULTILAYER

In [32]:
mlp = MLPClassifier(max_iter=3)

mlp.fit(x_treino,y_treino)

p3 = mlp.predict(x_teste)

view_score(y_teste, p3)

Pelo Accuracy:  83.11149165823409 %
Pelo Precision:  88.5049741437284 %
Pelo F1:  79.62945576315212 %

BASELINE
Pelo Accuracy:  45.61030244847478 %
Pelo Precision:  45.61030244847478 %
Pelo F1:  62.647081534102725 %


In [34]:
cross_validation(x,y,modelo=mlp)

Média Teste: 82.41 %
Entre : [82.35, 82.46] %


CLUSTERING

In [35]:
data_final = pd.concat([x,y], axis=1)

values = Normalizer().fit_transform(data_final.values)

km = KMeans(n_clusters=5,n_init=10,max_iter=300)
km_pred = km.fit_predict(values)

clus = data_final

clus['CLUSTER'] = km.labels_
descp = clus.groupby('CLUSTER')['QTDE_DIAS_ATIVO','QTDE_ATO_COBERTO_EXECUTADO']
descp = pd.DataFrame(descp.mean())
descp['CLIENTES'] = clus['CLUSTER'].value_counts()

print(descp)

         QTDE_DIAS_ATIVO  QTDE_ATO_COBERTO_EXECUTADO  CLIENTES
CLUSTER                                                       
0              41.820820                    0.957039     15130
1             225.981770                    4.508428     87383
2              97.297954                    2.268237     34257
3              10.970534                    0.216186      9774
4            1083.989597                    7.574297    238866


GRAFICO

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_final['QTDE_DIAS_ATIVO'],
                         y=data_final['QTDE_ATO_COBERTO_EXECUTADO'],
                         mode='markers',
                         marker=dict(color = km.labels_)
                         )
              )

In [ ]:
fig.show()